In [1]:
import numpy as np
import cv2
from keras.preprocessing import image
import time

import sys
import os
import glob
import re
import scipy
import tensorflow as tf
import align.detect_face



# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from keras.models import model_from_json



Using TensorFlow backend.


In [2]:
#opencv initialization

# face_cascade = cv2.CascadeClassifier('/home/topica/anaconda3/envs/workspace/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')

# facenet 
image_size=160
margin= 44
gpu_memory_fraction=1.0

def load_and_align_data(image_path, image_size,margin, gpu_memory_fraction):
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
    img = scipy.misc.imread(os.path.expanduser(image_path))
    img_size = np.asarray(img.shape)[0:2]
    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    if (len(bounding_boxes)==0):
        bb=0
        have_face = 0
    else:
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2 - bb[0], img_size[1])
        bb[3] = np.minimum(det[3]+margin/2 - bb[1], img_size[0])
        have_face = 1
    return bb,have_face

In [3]:
from keras.models import model_from_json
# model = model_from_json(open("/home/thaovu/tensorflow-101/model/facial_expression_model_structure.json", "r").read())
# model.load_weights('/home/thaovu/tensorflow-101/model/facial_expression_model_weights.h5') #load weights
model = model_from_json(open("/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.json", "r").read())
model.load_weights('/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.h5') #load weights

In [4]:
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [56]:
img_path = '/home/topica/Pictures/Webcam/fear9.jpg'
# # img = cv2.resize(img, (300, 400))
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [57]:
img = cv2.imread(img_path)
detect_face, have_face= load_and_align_data(img_path,image_size,margin,gpu_memory_fraction)
if (have_face!=0):
    detect_face = np.reshape(detect_face,(-1,4)) 
    for (x,y,w,h) in detect_face:
        detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
        detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48
        cv2.rectangle(img,(x,y),(x+w,y+h),(64,64,64),2) #highlight detected face
        
        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)

        img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

        #-----------------------------

        predictions = model.predict(img_pixels) #store probabilities of 7 expressions
        max_index = np.argmax(predictions[0])
        print(max_index)

        #background of expression list
        overlay = img.copy()
        opacity = 0.4
        cv2.rectangle(img,(x+w+15,y-25),(x+w+150,y+115),(64,64,64),cv2.FILLED)
        cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)

        #connect face and expressions
        cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
        cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)

        emotion = ""
        for i in range(len(predictions[0])):
            emotion = "%s %s%s" % (emotions[i], round(predictions[0][i]*100, 2), '%')

            """if i != max_index:
                color = (255,0,0)"""

            color = (255,0,0)

            cv2.putText(img, emotion, (int(x+w+15), int(y-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
# cv2.imshow('img',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite('thao_fear.png',img)

/home/topica/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


3


True

In [ ]:
faces

In [ ]:
preds = []
for (x,y,w,h) in faces:
    #if w > 130: #trick: ignore small faces
        #cv2.rectangle(img,(x,y),(x+w,y+h),(64,64,64),2) #highlight detected face

        detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
        detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)

        img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

        #------------------------------
        predictions = model.predict(img_pixels) #store probabilities of 7 expressions
        preds.append(predictions)
        print(len(preds[0]))
        print (predictions)
        max_index = np.argmax(predictions[0])
        print(max_index)

        #background of expression list
        overlay = img.copy()
        opacity = 0.4
        cv2.rectangle(img,(x+w+15,y-25),(x+w+150,y+115),(64,64,64),cv2.FILLED)
        cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)

        #connect face and expressions
        cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
        cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)

        emotion = ""
        for i in range(len(predictions[0])):
            emotion = "%s %s%s" % (emotions[i], round(predictions[0][i]*100, 2), '%')

            """if i != max_index:
                color = (255,0,0)"""

            color = (255,0,0)

            cv2.putText(img, emotion, (int(x+w+15), int(y-12+i*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
# cv2.imshow('img',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite('c15.png',img)